# Statistic Geospatial Model

In [2]:
import pandas as pd
import arcpy
import os
from arcpy.sa import *
from arcpy import env
import numpy as np
import math
from arcpy import env
from arcpy.sa import *

In [3]:
arcpy.env.workspace =r"D:\Maxim\masterthesis\YAK_TEST\YAK_TEST\Default.gdb"

In [4]:
rasters = arcpy.ListRasters(":.tif")
raster_list = arcpy.ListRasters("") 

In [5]:
print(raster_list)

['Fill_DGM_PRO1', 'Fill_Clip_DG1', 'Fill_DGM_PRO2', 'Fill_Clip_DG2', 'FlowDir_Fill1', 'Basin_FlowDi1', 'FlowDir_DEM', 'FlowAcc_Flow4', 'FlowAcc_DEM_Clip', 'Reclass_Flowacc', 'Watersh_Flow1', 'Watersh_Flow2', 'Watersh_Flow3', 'Watersh_Flow4', 'Watersh_Flow5', 'Watersh_Flow6', 'Watersh_Flow7', 'Reclass_Flow1', 'Reclass_Flowacc_yakou', 'Clip_PanThirdPoleLandCoverDa', 'Clip_PanThirdPoleSoilTexture', 'Clip_Clip_PanThirdPoleSoilTe', 'Clip_PanThirdPoleSoilTexture2', 'SoilTexture', 'Clip_PanThirdPoleLandCoverDa1', 'LandCover', 'PanThirdPoleVegetationFunction', 'PanThirdPoleVegetationFuncti1', 'DGM', 'DGM_ProjectRaster', 'DGM_YAKOU', 'Slope_yakou', 'Aspect_Slope1', 'PermafrostZonation', 'PermafrostZonation_ProjectRa', 'Permafrost_zonationindex', 'TEMPERATURE', 'Vegetation1_Clip', 'rster_stream', 'PanThirdPolePermafros_proj', 'Clip_PanThirdPolePermafrost_proj', 'Clip_PanThirdPolePermafrostZ1', 'Clip_PanThirdPolePermafrostZ5', 'C', 'LandCoverMapofPanThirdPole_C', 'Soil_texture_pro', 'calculate_ra

In [43]:
Slope=Raster("slope")

In [44]:
print("Slope")

Slope


Raster Normalisation

In [45]:
#call the raster 
Slope=Raster("Slope")
#calculating the normalisation
Slope_N= ((Slope - Slope.minimum)/(Slope.maximum - Slope.minimum))

In [54]:
#make the same calculation for all the other raster
Aspect=Raster("Aspect_Slope1")
Aspect_N= ((Aspect - Aspect.minimum)/(Aspect.maximum - Aspect.minimum))


DGM=Raster("DGM_YAKOU")
DGM_N = ((DGM - DGM.minimum)/(DGM.maximum - DGM.minimum))

In [55]:

Solar_radiation=Raster("AreaSol_DGM_1")
Solar_radiation_N= ((Solar_radiation - Solar_radiation.minimum)/(Solar_radiation.maximum - Solar_radiation.minimum))

The Weight

In [76]:
#find the path
file =('D:\Maxim\masterthesis\Probabilitys.xlsx')
newData = pd.read_excel(file)
newData

,slope,Aspect,solar radiation,Elevation,total
0,0.500,0.250,0.125,0.125,1
1,0.125,0.500,0.250,0.125,1
2,0.125,0.125,0.500,0.250,1
3,0.500,0.125,0.125,0.250,1
4,0.250,0.500,0.125,0.125,1
5,0.125,0.250,0.500,0.125,1
6,0.125,0.125,0.250,0.500,1
7,0.500,0.125,0.125,0.250,1
8,0.250,0.500,0.125,0.125,1
9,0.125,0.250,0.500,0.125,1


Calculating of the Active layer thinkness

In [91]:
#calculating of the different layer thinkness using the table of weight
ALT_liste=[]
for i,rows in newData.iterrows():
    ALT=(rows["slope"]*Slope_N + rows["Aspect"]*Aspect_N + rows["solar radiation"]*Solar_radiation_N + rows["Elevation"]*DGM_N)
    #add in the liste
    ALT_liste.append(ALT)
    ALT
    #save als tif file
    ALT.save(r"D:\Maxim\masterthesis\YAK_TEST\ALT"+str(i)+"tif")

In [93]:
#Normalisation of the 21 result
ALT_liste_N=[]
liste=[0,1,2,3,5,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for i in  liste:
    ALT_N = ( ALT_liste[i]) - ( ALT_liste[i].minimum)/ (( ALT_liste[i].maximum) - ( ALT_liste[i].minimum)) 
    #add in the  liste
    ALT_N
    ALT_liste_N.append( ALT_N)
    #save als tif file
    ALT_N.save(r"D:\Maxim\masterthesis\YAK_TEST\ALT_N" + str(i)+".tif") 

In [94]:
#Original file 
ALT_original=Raster("Clip_PanThirdPolePermafrostZ2")

In [98]:
#Normalisation of the Original file
ALT_original_N= Raster((ALT_original - ALT_original.minimum)/(ALT_original.maximum - ALT_original.minimum))
#save als tif file
ALT_original_N.save(r"D:\Maxim\masterthesis\YAK_TEST\YAK_TEST\Permafrost_original_N." + str(i)+".tif") 

Standard deviation(not obligatory)

In [106]:
#calculating the difference between the original Active layer and each active layer
DIFF=[]
liste=[0,1,2,3,5,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for i in  liste:
    diff= ALT_original - ALT_liste[i]
    DIFF.append(diff)
    diff.save(r"D:\Maxim\masterthesis\YAK_TEST\difference_." + str(i)+".tif") 

In [108]:
#Calculating the standard deviation using arcpy
Sd=[]
SD_mean =[]
for i in  liste:
    X= arcpy.sa.Statistics(DIFF[i], 3, 3, "standarddeviation", True)
    Sd.append(X)
    X.save(r"D:\Maxim\masterthesis\YAK_TEST\sd." + str(i)+".tif") 
    SD_mean.append(X.mean)

ROOT MEAN SQUARE ERROR

In [99]:
#Root mean square error 
RMSE =[]
for i in  liste:
    A = arcpy.sa.Arithmetic ( ALT_liste[i],ALT_original,"Minus", "FirstOf", "FirstOf")
    X = arcpy.sa.Arithmetic(A,A, "Multiply", "FirstOf", "FirstOf")
    rmse = np.square(X.mean)
    RMSE.append(rmse) 

In [101]:
#Root mean square of the rasters after the normalisation
RMSE_N=[]
for i in  liste:
    A = arcpy.sa.Arithmetic (ALT_liste_N[i], ALT_original_N,"Minus", "FirstOf", "FirstOf")
    X = arcpy.sa.Arithmetic(A,A, "Multiply", "FirstOf", "FirstOf")
    rmse = np.square(X.mean)
    RMSE_N.append(rmse) 

Creating a table

In [112]:
#Rows and colums
case=["case1","case2","case3","case4","case5","case6","case7","case8","case9","case10","case11","case12","case13","case14","case15","case16","case17","case18","case19","case20","caase21"]
columns=["RMSE","RMSE_N"]

In [113]:
d1 = {"case": case, "RMSE":RMSE, "RMSE_N" : RMSE_N }

In [114]:
#panda dataframe
df= pd.DataFrame(d1)
df.to_excel("D:\Maxim\masterthesis\YAK_TEST\Dataframe_case.xlsx") 

In [115]:
df

,case,RMSE,RMSE_N
0,case1,0.006048,0.037113
1,case2,0.003731,0.021437
2,case3,0.001274,0.016512
3,case4,0.006493,0.041030
4,case5,0.001220,0.024821
5,case6,0.005886,0.012971
6,case7,0.003647,0.052266
7,case8,0.006493,0.041030
8,case9,0.005886,0.024821
9,case10,0.001220,0.012971


Find the best case

In [116]:
df["RMSE_N"].idxmin()

18